This notebook demonstrates an example of generating Python code using inference [CodeLlama-7b-Instruct-hf](https://huggingface.co/meta-llama/CodeLlama-7b-Instruct-hf) model, followed by validating the generated code by executing it in sandbox on [Tracto.ai](https://tracto.ai/).


This notebook uses docker image cr.eu-north1.nebius.cloud/e00faee7vas5hpsh3s/solutions/torch:v3 that was created as

```
FROM ghcr.io/tractoai/notebook-kernel-default:2024-12-17-18-49-00-ceb64d083

USER root

RUN pip install torch transformers peft trl datasets
RUN pip install tractorun -U

USER 1000
```

In [1]:
import yt.wrapper as yt
from yt import type_info
import uuid

Prepare working directory on YTSaurus.

In [3]:
username = yt.get_user_name()
if yt.exists(f"//sys/users/{username}/@user_info/home_path"):
    home = yt.get(f"//sys/users/{username}/@user_info/home_path")
    working_dir = f"{home}/{uuid.uuid4().hex}"
else:
    working_dir = f"//tmp/examples/{uuid.uuid4().hex}"
yt.create("map_node", working_dir)
print(working_dir)

In [4]:
yt.config["pickling"]["dynamic_libraries"]["enable_auto_collection"] = False
yt.config["pickling"]["ignore_system_modules"] = True
yt.config["pickling"]["safe_stream_mode"] = False  # important to run vllm

Upload dataset from huggingface to YTSaurus table: task description, inputs, and outputs.

In [6]:
from datasets import load_dataset

MAX_SAMPLES = 50

dataset = load_dataset("deepmind/code_contests")

dataset_path = f"{working_dir}/dataset"

table_data = (
    {
        "index": index,
        "description": record["description"],
        "input": list(record["private_tests"]["input"] + record["generated_tests"]["input"])[:MAX_SAMPLES],
        "output": list(record["private_tests"]["output"] + record["generated_tests"]["output"])[:MAX_SAMPLES],
    }
    for index, record in enumerate(dataset["train"])
)

schema = yt.schema.TableSchema(strict=True)
schema.add_column("index", type_info.Int32)
schema.add_column("description", type_info.String)
schema.add_column("input", type_info.List[type_info.String])
schema.add_column("output", type_info.List[type_info.String])

yt.create("table", dataset_path, force=True, attributes={"schema": schema.to_yson_type()})
yt.write_table(dataset_path, table_data, table_writer={"max_row_weight": 128 * 1024 * 1024})

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [7]:
import os

hf_token = os.environ.get("YT_SECURE_VAULT_HF_TOKEN", "")
assert hf_token != "", "set HF token in kernel's secrets to use llama"

Generate solutions by CodeLlama-7b-Instruct-hf.

In [9]:
from typing import Iterable
import logging
import sys
import random


BATCH_SIZE = 150


@yt.aggregator
def bulk_inference(records: Iterable[dict[str, str]]) -> Iterable[dict[str, str]]:
    from vllm import LLM, SamplingParams

    os.environ["HF_TOKEN"] = hf_token

    # yt job have to write all logs to stderr
    vllm_logger = logging.getLogger("vllm")
    vllm_logger.handlers.clear()
    vllm_logger.addHandler(logging.StreamHandler(sys.stderr))

    llm = LLM(model="meta-llama/CodeLlama-7b-Instruct-hf", tensor_parallel_size=1, trust_remote_code=True)
    sampling_params = SamplingParams(
        temperature=0.6,
        top_p=0.9,
        max_tokens=5000,
    )

    def generate(records_batch: list[dict]):
        conversations = [
            [
                {
                    "role": "system",
                    "content": "You are an AI assistant that generates Python code. Always write Python code that reads input from stdin and writes output to stdout. Ensure that all indentation is correct. Your responses must contain only ready-to-run Python code, without any explanations, comments, or extra text.",
                },
                {
                    "role": "user",
                    "content": r["description"],
                },
            ] for r in records_batch
        ]
        outputs = llm.chat(
            messages=conversations,
            sampling_params=sampling_params,
        )
        return outputs

    batch = []
    for record in records:
        batch.append(record)
        if len(batch) < BATCH_SIZE:
            continue
        outputs = generate(batch)
        for r, output in zip(batch, outputs):
            yield {
                "index": r["index"],
                "description": r["description"],
                "input": r["input"],
                "output": r["output"],
                "code": output.outputs[0].text.strip(" "),
            }
        batch = []
    if batch:
        outputs = generate(batch)
        for r, output in zip(batch, outputs):
            yield {
                "index": r["index"],
                "description": r["description"],
                "input": r["input"],
                "output": r["output"],
                "code": output.outputs[0].text.strip(" "),
            }

In [10]:
inference_path = f"{working_dir}/inference"

schema = yt.schema.TableSchema(strict=True)
schema.add_column("index", type_info.Int32)
schema.add_column("description", type_info.String)
schema.add_column("input", type_info.List[type_info.String])
schema.add_column("output", type_info.List[type_info.String])
schema.add_column("code", type_info.String)

yt.create("table", inference_path, force=True, attributes={"schema": schema.to_yson_type()})


yt.run_map(
    bulk_inference,
    dataset_path,
    inference_path,
    job_count=16,
    spec={
        "pool_trees": ["gpu_100"],
        "mapper": {
            "gpu_limit": 1,
            "memory_limit": 32212254720,
            "cpu_limit": 2,
        },
        "job_io": {
            "table_writer": {
                "max_row_weight": 128 * 1024 * 1024,
            },
        },
    },
)

2025-02-07 23:11:11,794	INFO	Operation started: https://playground.yt.nebius.yt/playground/operations/3deaffea-e4f5bb74-270703e8-1781d41e/details


2025-02-07 23:11:11,832	INFO	( 0 min) operation 3deaffea-e4f5bb74-270703e8-1781d41e starting


2025-02-07 23:11:12,357	INFO	( 0 min) operation 3deaffea-e4f5bb74-270703e8-1781d41e initializing


2025-02-07 23:11:13,444	INFO	( 0 min) Unrecognized spec: {'enable_partitioned_data_balancing': false, 'mapper': {'title': 'bulk_inference'}}


2025-02-07 23:11:15,147	INFO	( 0 min) operation 3deaffea-e4f5bb74-270703e8-1781d41e: running=0     completed=0     pending=16    failed=0     aborted=0     lost=0     total=16    blocked=0    


...

2025-02-07 23:20:21,211	INFO	( 9 min) operation 3deaffea-e4f5bb74-270703e8-1781d41e: running=6     completed=13    pending=0     failed=0     aborted=0     lost=0     total=19    blocked=0    


2025-02-07 23:20:51,537	INFO	( 9 min) operation 3deaffea-e4f5bb74-270703e8-1781d41e: running=4     completed=14    pending=0     failed=0     aborted=1     lost=0     total=18    blocked=0    


2025-02-07 23:21:06,692	INFO	( 9 min) operation 3deaffea-e4f5bb74-270703e8-1781d41e: running=2     completed=15    pending=0     failed=0     aborted=2     lost=0     total=17    blocked=0    


2025-02-07 23:21:21,826	INFO	(10 min) operation 3deaffea-e4f5bb74-270703e8-1781d41e completed


2025-02-07 23:21:21,868	INFO	(10 min) Alerts: {'low_cpu_usage': {'code': 1, 'message': "Average CPU usage of some of your job types is significantly lower than requested 'cpu_limit'. Consider decreasing cpu_limit in spec of your operation", 'attributes': {'pid': 1, 'tid': 9389937110908861621, 'thread': 'Controller:14', 'fid': 18446213275216545196, 'host': 'man0-0461.hw.nebius.yt', 'datetime': '2025-02-07T23:21:20.559415Z', 'trace_id': '47b668fc-901cb8db-9743acb1-4464732d', 'span_id': 11011441638619648255}, 'inner_errors': [{'code': 1, 'message': 'Jobs of task "map" use 30.65% of requested cpu limit', 'attributes': {'pid': 1, 'tid': 9389937110908861621, 'thread': 'Controller:14', 'fid': 18446213275216545196, 'host': 'man0-0461.hw.nebius.yt', 'datetime': '2025-02-07T23:21:20.559405Z', 'trace_id': '47b668fc-901cb8db-9743acb1-4464732d', 'span_id': 11011441638619648255, 'cpu_time': 5029037, 'cpu_limit': 2.0, 'exec_time': 8204616}}]}}


Let's run the code in parallel and validate the results. We'll limit memory consumption for the tested code using prlimit.
In this example, we do not isolate the llm-generated code from the controlling system at the io-level, but we limit RAM consumption.

In [12]:
import subprocess
import sys


RUN_TIMEOUT = 10
CODE_IO_LIMIT = 2000
MEM_LIMIT = 32212254720


@yt.aggregator
def validate_code(records: Iterable[dict[str, str]]) -> Iterable[dict[str, str]]:
    for record in records:
        for inp, expected in zip(record["input"], record["output"]):
            print(f"Run code for {record['index']}", file=sys.stderr)
            try:
                process = subprocess.run(
                    ["prlimit", f"--as={int(MEM_LIMIT * 0.5)}", "python3", "-c", record["code"]],
                    input=inp,
                    text=True,
                    capture_output=True,
                    timeout=RUN_TIMEOUT,
                )
            except subprocess.TimeoutExpired:
                print(f"Execution failed: timeout", file=sys.stderr)
                yield {
                    "index": record["index"],
                    "description": record["description"],
                    "input": record["input"],
                    "output": record["output"],
                    "code": record["code"],
                    "result_stdout": "<TIMEOUT>",
                    "result_stderr": "<TIMEOUT>",
                    "exitcode": -1,
                    "match_expected": False,
                }
                continue
            if process.returncode:
                print(f"Execution failed: {process.stderr}", file=sys.stderr)

            yield {
                "index": record["index"],
                "description": record["description"],
                "input": record["input"],
                "output": record["output"],
                "code": record["code"],
                "result_stdout": process.stdout[:CODE_IO_LIMIT],
                "result_stderr": process.stderr[:CODE_IO_LIMIT],
                "exitcode": process.returncode,
                "match_expected": process.stdout == expected,
            }

In [13]:
code_validation_path = f"{working_dir}/code_validation"

schema = yt.schema.TableSchema(strict=True)
schema.add_column("index", type_info.Int32)
schema.add_column("description", type_info.String)
schema.add_column("input", type_info.List[type_info.String])
schema.add_column("output", type_info.List[type_info.String])
schema.add_column("code", type_info.String)
schema.add_column("result_stdout", type_info.String)
schema.add_column("result_stderr", type_info.String)
schema.add_column("exitcode", type_info.Int8)
schema.add_column("match_expected", type_info.Bool)

yt.create("table", code_validation_path, force=True, attributes={"schema": schema.to_yson_type()})

yt.run_map(
    validate_code,
    inference_path,
    code_validation_path,
    job_count=256,
    spec={
        "mapper": {
            "memory_limit": MEM_LIMIT,
            "cpu_limit": 2,
        },
        "job_io": {
            "table_writer": {
                "max_row_weight": 128 * 1024 * 1024,
            },
        },
    },
)

2025-02-07 23:21:23,823	INFO	Operation started: https://playground.yt.nebius.yt/playground/operations/af333775-a51d7aeb-270703e8-aa4132b1/details


2025-02-07 23:21:23,845	INFO	( 0 min) operation af333775-a51d7aeb-270703e8-aa4132b1 starting


2025-02-07 23:21:24,374	INFO	( 0 min) operation af333775-a51d7aeb-270703e8-aa4132b1 initializing


2025-02-07 23:21:25,983	INFO	( 0 min) Unrecognized spec: {'enable_partitioned_data_balancing': false, 'mapper': {'title': 'validate_code'}}


2025-02-07 23:21:26,004	INFO	( 0 min) operation af333775-a51d7aeb-270703e8-aa4132b1: running=0     completed=0     pending=256   failed=0     aborted=0     lost=0     total=256   blocked=0    


...

2025-02-08 00:06:29,439	INFO	(45 min) operation af333775-a51d7aeb-270703e8-aa4132b1: running=5     completed=252   pending=0     failed=0     aborted=25    lost=0     total=257   blocked=0    


2025-02-08 00:08:25,556	INFO	(47 min) operation af333775-a51d7aeb-270703e8-aa4132b1: running=4     completed=253   pending=0     failed=0     aborted=25    lost=0     total=257   blocked=0    


2025-02-08 00:12:53,084	INFO	(51 min) operation af333775-a51d7aeb-270703e8-aa4132b1: running=2     completed=254   pending=0     failed=0     aborted=26    lost=0     total=256   blocked=0    


2025-02-08 00:12:58,138	INFO	(51 min) operation af333775-a51d7aeb-270703e8-aa4132b1: running=1     completed=255   pending=0     failed=0     aborted=26    lost=0     total=256   blocked=0    


2025-02-08 00:14:49,170	INFO	(53 min) operation af333775-a51d7aeb-270703e8-aa4132b1 completed
